# SCIBERT in HuggingFace

In [1]:
# run this script to import the variables and settings from ipynb_util.py
%run ipynb_util.py

In [2]:
# select dataset
from enum import Enum

class DatasetType(Enum):
    """Enum for the dataset type."""

    """Zora + OSDG dataset"""
    ZO_UP = "zo_up"
    """SwissText Shared Task 1 dataset (Zurich NLP)"""
    SWISSTEXT_SHARED_TASK1 = "swisstext_shared_task1"

    @property
    def

DATASET_TYPE = DatasetType.ZO_UP

In [3]:
from datasets import load_dataset, Features, Value, ClassLabel

# load the dataset
# note: if you don't have the data in the folder, use the download-data.sh script

match DATASET_TYPE:
    case DatasetType.ZO_UP:
        sdgs = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "non-relevant"]
        featuers = Features({"sdg": ClassLabel(num_classes=len(sdgs), names=sdgs), "abstract": Value("string")})
        dataset = load_dataset("csv", data_files=str(DATA_DIR_PATH / "zo_up.csv"))
        dataset = dataset.rename_columns({"sdg": "SDG", "abstract": "ABSTRACT"}).class_encode_column("SDG")
        dataset = dataset["train"].train_test_split(test_size=0.3, stratify_by_column="SDG", seed=SEED)
    case DatasetType.SWISSTEXT_SHARED_TASK1:
        dataset = load_dataset("json", data_files=str(DATA_DIR_PATH / "swisstext-2024-sharedtask" / "task1-train.jsonl"))
        dataset = dataset["train"].train_test_split(test_size=0.3, seed=SEED)

print(dataset["train"].features)
example = dataset["train"][0]
print("Example instance:\t", example)

labels = set(dataset["train"]["SDG"])
# identity because labels are already ids and vice-versa
id2label = {i: dataset["train"].features["SDG"].int2str(i) for i in range(len(labels))}
label2id = {dataset["train"].features["SDG"].int2str(i): i for i in range(len(labels))}

labels

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [4]:
from transformers import AutoTokenizer

# base model
HF_MODEL_NAME = "allenai/scibert_scivocab_uncased"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-{DATASET_TYPE.value}"

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)

def preprocess_data(instances):
    match DATASET_TYPE:
        case DatasetType.SWISSTEXT_SHARED_TASK1:
            # take a batch of titles and abstracts and concat them
            titles = instances["TITLE"]
            abstracts = instances["ABSTRACT"]
            texts = [f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]
        case DatasetType.ZO_UP:
            texts = instances["ABSTRACT"]

    # encode
    encoding = tokenizer(texts, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    # add labels
    encoding["label"] = torch.tensor([label for label in instances["SDG"]])

    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


In [5]:
example = encoded_dataset["train"][0]
print("Example instance:\t", example)

tokenizer.decode(example["input_ids"])

Example instance:	 {'input_ids': tensor([  102,   111,   705,  7834,   147,  5901,   767,  4427,  6703,   111,
         1232, 11812,  2393,   190,   106, 11812,  5612,  3850,   422,  5468,
         3441,   205,   111,  7021,   579, 20665, 30113,   267,  6703,   214,
        17162,   422,   111, 11812, 22668,  2393,   214,   572, 11451,   131,
          952, 17898,   205,  1491,   301,  4696,   556,   422,   461,   241,
         2404,  2011,   422,   137,  4095,   147,   626, 26503,   579,  1252,
          794, 21830,  6820,   205,  5555,   883,  9945,   168,   547,  2910,
         7909,   422,   698,  2764,  1883,  2208,   137,  1883,  3982,   205,
          111,  5612,  3850,   241,   106, 23398,  1004, 28364,   422,  3246,
          185,  5414,   355,   111, 15304,  3779,   198, 17771,   256,   205,
          103,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

'[CLS] the first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. the drive - shafts were replaced by wires, the huge steam engine by dozens of small motors. factories spread out, there was natural light, and room to use ceiling - slung cranes. workers had responsibility for their own machines, they needed better training and better pay. the electric motor was a wonderful invention, once we changed all the everyday details that surrounded it. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 8
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    #save_steps=10,
    #eval_steps=10,
    #logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=15,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.543400,1.996294,0.568266,0.573122,0.568266,0.531218
2,1.415400,1.264565,0.678967,0.669639,0.678967,0.664313
3,0.716700,1.086663,0.708487,0.698641,0.708487,0.695318
4,0.387600,1.054023,0.719557,0.716750,0.719557,0.711743
5,0.194600,1.205441,0.726937,0.730698,0.726937,0.718986
6,0.112000,1.278205,0.730627,0.728600,0.730627,0.724242
7,0.067900,1.399384,0.704797,0.705396,0.704797,0.698122


/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

## Evaluation

In [ ]:
trainer.evaluate()

/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.0742533206939697,
 'eval_accuracy': 0.5503875968992248,
 'eval_precision': 0.6207630338957288,
 'eval_recall': 0.5503875968992248,
 'eval_f1': 0.5498825849667871,
 'eval_runtime': 4.3855,
 'eval_samples_per_second': 29.415,
 'eval_steps_per_second': 3.876,
 'epoch': 20.0}

## Inference

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

text = "This is a test text about climate change."

MODEL_PATH = f"{CHECKPOINT_PATH}/{MODEL_NAME}/checkpoint-532"
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    id2label={i: label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)}
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.eval()
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(model.device) for k, v in encoding.items()}

outputs = model(**encoding)

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 18])

In [ ]:
softmax = torch.nn.Softmax()
probs = softmax(logits.squeeze().cpu())

# for multi-label classification, we need to threshold the probabilities
"""
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
predicted_labels = [id2label(i) for i in np.where(predictions == 1)]
print(predicted_labels)
"""
# for single-label classification, we can take the argmax
print(probs)
predictions = probs.argmax(-1)
predicted_label = id2label[predictions.int().item()]
print(predicted_label)

tensor([1.4732e-03, 8.4256e-04, 1.2375e-03, 1.2201e-03, 1.0317e-03, 1.0760e-03,
        1.3087e-03, 1.3920e-03, 2.3354e-03, 7.2098e-04, 6.6897e-04, 8.4898e-04,
        8.5957e-04, 9.7841e-01, 1.9451e-03, 1.9424e-03, 1.5772e-03, 1.1073e-03],
       grad_fn=<SoftmaxBackward0>)
13


In [ ]:
import gc

def my_reset(*varnames):
    """
    Resets global variables and majority of CUDA memory. Only works in Jupyter.

    varnames are what you want to keep
    """
    try:
        del model
    except NameError:
        pass
    gc.collect()
    with torch.no_grad():
        torch.cuda.ipc_collect()
        torch.cuda.empty_cache()

    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

# my_reset()

/tmp/ipykernel_27454/3741609671.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic("reset")


Nothing done.


### Investigating sklearn precision warning

In [ ]:
p = torch.ones((86, 18)).argmax(-1)
p[:10] = torch.Tensor([i for i in range(10)])
t = torch.ones(86, dtype=torch.long)
t[:18] = torch.Tensor([i for i in range(18)])

# no Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(10)])

(0.9873459873459873, 0.1282051282051282, 0.12816755893678972, None)

In [ ]:
# Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(11)])

/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9748479368732533, 0.12658227848101267, 0.12654518477303286, None)